In [18]:
#!pip install plotly

# 1. 라이브러리 설치

In [19]:
import pandas as pd
import os
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
warnings.filterwarnings(action='ignore')
%matplotlib inline

# 그래프 한글폰트 해결
from matplotlib import font_manager,rc
font_location="c:/windows/fonts/malgun.ttf"
font_name=font_manager.FontProperties(fname=font_location).get_name()
rc('font',family=font_name)

# 마이너스 부호 깨짐 해결
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

# 2. 사용자 설정

2-1. default 경로 지정

In [20]:
default_dir = 'C:/19Project/05.TRV/01.Pred_Demand/'

2-2. 파일 경로 설정

In [21]:
data_dir = default_dir + '0. 데이터/output_20211006_1/'

# 3. 데이터 불러오기

3-1. 파일명 설정

In [22]:
data_file = '운행데이터_전처리 완료.csv'
addr_file = 'tmp_addr_list.csv'
time_file = 'tmp_time_list.csv'

3-2. 디렉터리 생성

In [23]:
now = datetime.datetime.now().date().strftime('%Y%m%d')

num = 1

eda_dir = default_dir + '2. EDA_' + now

while os.path.exists(eda_dir + '_' + str(num)):
    num += 1
    
eda_dir_num = eda_dir + '_' + str(num) + '/'
    
os.makedirs(os.path.join(eda_dir_num))

3-3. 전처리 완료 데이터, 구역 리스트, 시각 리스트 불러오기

In [24]:
data = pd.read_csv(data_dir + data_file, engine = 'python', encoding='cp949', converters = {1: str}, parse_dates=[3, 7, 11, 18, 23, 28])
data.columns = ['seq', '차량 번호', '차량 소속', '콜접수 일자', '콜접수 시각', '콜접수 분', '장애 유형', '승차 일자', 
                '승차 시각', '승차 분', '승차 구역명', '하차 일자', '하차 시각', '하차 분', '하차 구역명', '운행 거리', '픽업 거리', 
                '이동 구분', '콜접수 일시', '콜접수 연월', '콜접수 요일', '콜접수 일자 특성', '콜접수 일자 특성 구분', 
                '승차 일시', '승차 연월', '승차 요일', '승차 일자 특성', '승차 일자 특성 구분', '하차 일시', '대기 시간']

addr_list = pd.read_csv(data_dir + addr_file, engine = 'python')
time_list = pd.read_csv(data_dir + time_file, engine = 'python')

3-4. 요일, 일자 특성 정렬기준 설정

In [25]:
weekday_mapping = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
data['승차 요일'] = pd.Categorical(data['승차 요일'], weekday_mapping.values())
data['콜접수 요일'] = pd.Categorical(data['콜접수 요일'], weekday_mapping.values())

calendar_mapping = {1: '평일', 2: '주말', 3: '공휴일', 4: '휴일 전후일', 5: '연휴(3일연속)', 6: '징검다리 평일', 7:'명절(설, 추석)'}
data['승차 일자 특성'] = data['승차 일자 특성'].map(calendar_mapping)
data['승차 일자 특성'] = pd.Categorical(data['승차 일자 특성'], calendar_mapping.values())
data['콜접수 일자 특성'] = data['콜접수 일자 특성'].map(calendar_mapping)
data['콜접수 일자 특성'] = pd.Categorical(data['콜접수 일자 특성'], calendar_mapping.values())

# 4. 함수 생성

In [26]:
# 이동구분별 운행 거리 분석
def distance_result(df):
    # 이동 구분 리스트 생성
    run_type = ['전체'] + df['이동 구분'].unique().tolist()
    # 이동 구분별 분석 실행
    for t in run_type:
        if t != '전체':
            tmp = df[df['이동 구분'] == t]
        elif t == '전체':
            tmp = df
        # summary
        distance_summary = pd.DataFrame(tmp['운행 거리'].describe())
        distance_summary.index = ['count', 'mean', 'std', 'min', 'q1', 'median', 'q3', 'max']
        distance_summary.columns = ['value']   
        # iqr, uif, outlier
        distance_q1 = distance_summary.loc['q1']['value']
        distance_q3 = distance_summary.loc['q3']['value']
        distance_iqr = distance_q3 - distance_q1
        distance_uif = distance_q3 + (1.5 * distance_iqr)
        distance_outlier = len(tmp[tmp['운행 거리'] > distance_uif])
        # make summary table
        distance_table = distance_summary.loc[['count', 'min', 'q1', 'median', 'mean', 'q3', 'max']]
        distance_add = pd.DataFrame([distance_iqr, distance_uif, distance_outlier])
        distance_add.index = ['IQR', 'UIF', 'outlier']
        distance_add.columns = ['value']
        distance_table = pd.concat([distance_table, distance_add], axis = 0)
        # save
        table_name = 'EDA_운행 거리_' + t + '_Summary.csv'
        distance_table.to_csv(eda_dir_num + table_name, encoding = 'MS949')
        print('[운행 거리 분석_' + t + '] \n' + eda_dir_num + table_name)
        # plot
        plt.rcParams["figure.figsize"] = (10,10)
        f, (ax_box, ax_hist) = plt.subplots(1, 2, sharex = 'col', gridspec_kw={"width_ratios": (.3, .7)})
        sns.boxplot(tmp['운행 거리'], orient = 'v', ax = ax_box)
        sns.distplot(tmp['운행 거리'],  vertical = True, ax = ax_hist)
        ax_box.set_ylabel('운행 거리', labelpad = 20, fontsize = 13)
        ax_hist.set(ylabel='')
        ax_box.set_title('[특별교통수단 운영 현황 - 운행 거리_' + t + ']', pad = 20, fontsize = 20, loc = 'left')   
        plot_name = 'EDA_운행 거리_' + t + '_Plot.png'
        plt.savefig(eda_dir_num + plot_name)
        print(eda_dir_num + plot_name +  '\n')
        plt.close()
        

# 장애유형별 운행 수 분석
def disorder_table(df):
    disorder_table = pd.DataFrame(df['장애 유형'].value_counts())
    disorder_table.columns = ['운행 수']
    disorder_table.sort_values('운행 수', ascending = False, inplace = True)
    table_name = 'EDA_장애 유형별 운행 수 Table.csv'
    disorder_table.to_csv(eda_dir_num + table_name, encoding = 'MS949')
    print('[장애 유형 분석] \n' + eda_dir_num + table_name + '\n')
    

# 차량 번호별 운행 분석    
def car_table(df):
    # 차량 소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 차량소속별 분석 실행
    for t in car_type:
        if t != '전체':
            tmp = df[df['차량 소속'] == t]
        elif t == '전체':
            tmp = df
        # 차량 번호별 운행 수
        car_freq = pd.DataFrame(tmp.groupby(['차량 소속','차량 번호'])['seq'].count())
        car_freq.reset_index(inplace = True)
        car_freq.columns = ['차량 소속', '차량 번호', '운행 수']
        # 차량 번호별 누적 운행 거리
        car_distance = pd.DataFrame(tmp.groupby(['차량 소속','차량 번호'])['운행 거리'].sum())
        car_distance.reset_index(inplace = True)
        car_distance.columns = ['차량 소속', '차량 번호', '누적 운행 거리']
        # 차량 번호별 누적 픽업 거리
        car_pickup = pd.DataFrame(tmp.groupby(['차량 소속', '차량 번호'])['픽업 거리'].sum())
        car_pickup.reset_index(inplace = True)
        car_pickup.columns = ['차량 소속', '차량 번호', '누적 픽업 거리']
        # 테이블 합치기
        car_table = car_freq.merge(car_distance, on = ['차량 소속', '차량 번호'], how = 'outer')
        car_table = car_table.merge(car_pickup, on = ['차량 소속', '차량 번호'], how = 'outer')
        # 회당 거리 계산
        car_table['회당 운행 거리'] = car_table['누적 운행 거리'] / car_table['운행 수']
        car_table['회당 픽업 거리'] = car_table['누적 픽업 거리'] / car_table['운행 수']
        # 운행 분석 테이블 정렬
        car_table.sort_values('운행 수', ascending = False, inplace = True)
        car_table = car_table[['차량 소속', '차량 번호', '운행 수', '누적 운행 거리', '회당 운행 거리', '누적 픽업 거리', '회당 픽업 거리']]
        # 픽업 거리 데이터가 없는 경우 픽업 거리 컬럼 삭제
        if len(car_table[car_table['누적 픽업 거리'] == 0]) == len(car_table):
            car_table.drop(['누적 픽업 거리', '회당 픽업 거리'], axis = 1, inplace = True)
        # save
        table_name = 'EDA_차량 번호별 운행 내역_' + t + '_Table.csv'
        car_table.to_csv(eda_dir_num + table_name, index = False, encoding = 'MS949')
        print('[차량 번호별 운행 내역 분석_' + t+ '] \n' + eda_dir_num + table_name + '\n')
        
        
# 승차 연월별 운행 내역 분석 (차량 소속 구분)
def month_result(df):
    # 차량 소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 승차 연월 리스트 생성
    ym_list = pd.DataFrame(list(df['승차 연월'].sort_values().unique()))
    ym_list.columns = ['승차 연월']
    ym_list.reset_index(drop = True, inplace = True)
    # 차량 소속별 분석 실행
    for t in car_type:
        if t != '전체':
            tmp = df[df['차량 소속'] == t]
        elif t == '전체':
            tmp = df
        # 연월별 운행 수
        month_freq = pd.DataFrame(tmp['승차 연월'].value_counts())
        month_freq = month_freq.reset_index()
        month_freq.columns = ['승차 연월','운행 수']
        # 연월별 운행 거리
        month_distance = pd.DataFrame(tmp.groupby(['승차 연월'])['운행 거리'].sum())
        month_distance = month_distance.reset_index()
        month_distance.columns = ['승차 연월', '누적 운행 거리']
        # 연월별 차량 수
        car_n = tmp[['승차 연월', '차량 번호']]
        car_n = car_n.drop_duplicates()
        car_table = pd.DataFrame(car_n['승차 연월'].value_counts())
        car_table.reset_index(inplace = True)
        car_table.columns = ['승차 연월', '차량 수']
        car_table['index'] = range(0, len(car_table))
        car_table.set_index('index', inplace = True)
        # 연월별 운행 내역 분석 table 생성
        month_table = month_freq.merge(month_distance, on = '승차 연월', how = 'outer')
        month_table = month_table.merge(car_table, on = '승차 연월', how = 'outer')
        month_table.sort_values('승차 연월', inplace = True)
        month_table.reset_index(drop = True, inplace = True)
        # 누락된 연월 추가 (승차 연월 리스트 사용)
        month_table = ym_list.merge(month_table, on = '승차 연월', how = 'left')
        month_table['운행 수'] = month_table['운행 수'].fillna(0)
        month_table['누적 운행 거리'] = month_table['누적 운행 거리'].fillna(0)
        month_table['차량 수'] = month_table['차량 수'].fillna(0).astype(int)
        # save
        table_name = 'EDA_승차 연월별 운행 분석_' + t + '_Table.csv'
        month_table.to_csv(eda_dir_num + table_name, index = False, encoding = 'MS949')
        print('[승차 연월별 운행 분석_' + t + '] \n' + eda_dir_num + table_name)
        # plot
        plt.figure(figsize = (20,15))
        ax1 = month_table['운행 수'].plot(kind = 'bar', color = 'C0')
        plt.grid()
        ax2 = ax1.twinx()
        ax2.plot(month_table['누적 운행 거리'], color = 'C1', label = '누적 운행거리(km)', linewidth = 2.0)
        plt.legend(fontsize = 20)
        plt.xticks(range(0, len(month_table)) ,month_table['승차 연월'])
        ax1.tick_params(axis = 'x', labelsize = 15)
        ax1.tick_params(axis = 'y', labelsize = 15)
        ax2.tick_params(axis = 'y', labelsize = 15)
        ax1.set_ylabel('운행 수', fontsize = 20, labelpad = 20)
        ax2.set_ylabel('운행 거리', fontsize = 20, labelpad = 20)
        ax1.set_ylim(0, month_table['운행 수'].max() * 1.15)
        ax2.set_ylim(0, month_table['누적 운행 거리'].max() * 1.2)
        for index, row in month_table.iterrows():
            ax1.text(row.name, row['운행 수'] + (month_table['누적 운행 거리'].max() / (25.4 * 15)), 
                row['차량 수'], color = 'black', ha = "center", fontsize = 20)        
        plt.title('[특별교통수단 운영 현황 - ' + '승차 연월별 운행 내역_' + t + ']', fontsize = 30, pad = 20, loc = 'left')
        plot_name = 'EDA_승차 연월별 운행 분석_' + t + '_Plot.png'
        plt.savefig(eda_dir_num + plot_name)
        print(eda_dir_num + plot_name + '\n')
        plt.close()
        

# 승차 시각별, 요일별, 일자 특성별 운행 내역 분석
def run_result(df):
    # 차량 소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 분석 컬럼 리스트 생성 (승차 시각, 승차 요일, 승차 일자 특성)
    col_list = ['승차 시각', '승차 요일', '승차 일자 특성']
    # 컬럼별 분석 실행
    for c in col_list:
        # 해당 컬럼 전체 리스트 생성
        col_tmp = pd.DataFrame(list(df[c].sort_values().unique()))
        col_tmp.columns = [c]
        col_tmp.reset_index(drop = True, inplace = True)
        # 차량소속별 분석 실행
        for t in car_type:
            if t != '전체':
                tmp = df[df['차량 소속'] == t]
            elif t == '전체':
                tmp = df
            # 운행 수
            data_freq = pd.DataFrame(tmp[c].value_counts())
            data_freq.reset_index(inplace = True)
            data_freq.columns = [c, '운행 수']
            # 운행 거리
            data_distance = pd.DataFrame(tmp.groupby(c)['운행 거리'].sum())
            data_distance.reset_index(inplace = True)
            data_distance.columns = [c, '누적 운행 거리']
            # table 생성
            data_table = data_freq.merge(data_distance, on = c, how = 'outer')
            data_table.sort_values(c, inplace = True)
            data_table.reset_index(drop = True, inplace = True)
            # 누락된 항목 추가
            data_table = col_tmp.merge(data_table, on = c, how = 'left')
            data_table['운행 수'] = data_table['운행 수'].fillna(0)
            data_table['누적 운행 거리'] = data_table['누적 운행 거리'].fillna(0)
            # save
            table_name = 'EDA_' + c + '별 운행 분석_' + t + '_Table.csv'
            data_table.to_csv(eda_dir_num + table_name, index = False, encoding = 'MS949')
            print('[' + c + '별 운행 분석_' + t + '] \n' + eda_dir_num + table_name)
            # plot
            plt.figure(figsize = (20,15))
            ax1 = data_table['운행 수'].plot(kind = 'bar', color = 'C0')
            plt.grid()
            ax2 = ax1.twinx()
            ax2.plot(data_table['누적 운행 거리'], color = 'C1', label = '누적 운행 거리(km)', linewidth = 2.0)
            plt.legend(fontsize = 20)
            plt.xticks(range(0, len(data_table)), data_table[c])
            ax1.tick_params(axis = 'x', labelsize = 15)
            ax1.tick_params(axis = 'y', labelsize = 15)
            ax2.tick_params(axis = 'y', labelsize = 15)
            ax1.set_ylabel('운행 수', fontsize = 20, labelpad = 40)
            ax2.set_ylabel('운행 거리', fontsize = 20, labelpad = 40)
            ax1.set_ylim(0, data_table['운행 수'].max() * 1.15)
            ax2.set_ylim(0, data_table['누적 운행 거리'].max() * 1.2)
            plt.title('[특별교통수단 운영 현황 - ' + c +'별 운행 내역_' + t + ']', fontsize = 30, pad = 20, loc = 'left')
            plot_name = 'EDA_' + c + '별 운행 분석_' + t + '_Plot.png'
            plt.savefig(eda_dir_num + plot_name)
            print(eda_dir_num + plot_name + '\n')
            plt.close()

            
# 요일, 일자 특성별 운행 수 BOXPLOT, SUMMARY
def box_result(df):
    # 차량 소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 분석 컬럼 리스트 생성
    col_list = ['승차 요일', '승차 일자 특성']
    # 컬럼별 분석 실행
    for c in col_list:
        # 차량소속별 분석 실행
        for t in car_type:
            if t != '전체':
                tmp = df[df['차량 소속'] == t]
            elif t == '전체':
                tmp = df
            # 일별 운행 수 집계
            day_table = pd.DataFrame(tmp.groupby(['승차 일자', c])['seq'].count())
            day_table.reset_index(inplace = True)
            day_table.columns = ['승차 일자', c, '운행 수']
            c_unique = day_table[c].sort_values().unique()
            # summary
            summary_result = pd.DataFrame()
            for u in c_unique:
                tmp_table = day_table[day_table[c] == u]
                tmp_summary = pd.DataFrame(tmp_table['운행 수'].describe())
                tmp_summary.columns = ['운행 수_' + u]
                summary_result = pd.concat([summary_result, tmp_summary], axis = 1)
            summary_result = summary_result.loc[['count', 'min', '25%', '50%', 'mean', '75%', 'max']]
            summary_result.index = ['count', 'min', 'q1', 'median', 'mean', 'q3', 'max']
            # save
            table_name = 'EDA_' + c + '별 일별 운행 수 특성 분석_' + t + '_Summary.csv'
            summary_result.to_csv(eda_dir_num + table_name, encoding = 'MS949')
            print('[' + c + '별 운행 수 특성 분석_' + t + '] \n' + eda_dir_num + table_name)       
            # 그래프 색 설정 (평일, 휴일 구분)
            week_pal = {"월": "C0", "화": "C0", "수": "C0", "목": "C0", "금": "C0", "토": "C1", "일": "C1"}
            calendar_pal = {"평일" : "C0", "주말" : "C1", "공휴일" : "C1", "휴일 전후일" : "C0", "연휴(3일연속)" : "C1", 
                            "징검다리 평일" : "C0",  "명절(설, 추석)" : "C1"}        
            if c == '승차 요일':
                pal = week_pal
            elif c == '승차 일자 특성':
                pal = calendar_pal
            # boxplot
            plt.figure(figsize = (20,15))
            sns.boxplot(x = c, y = '운행 수', data = day_table, palette = pal)
            plt.xticks(fontsize = 17)
            plt.yticks(fontsize = 20)
            plt.xlabel('')
            plt.ylabel('')
            plt.title('[특별교통수단 운영 현황 - ' + c +'별 일별 운행 수 특성_' + t + ']', fontsize = 30, pad = 20, loc = 'left')
            plot_name = 'EDA_' + c + '별 일별 운행 수 특성 분석_' + t + '_BoxPlot.png'
            plt.savefig(eda_dir_num + plot_name)
            print(eda_dir_num + plot_name + '\n')
            plt.close()
        

# 승차구역별 픽업거리 분석        
def pickup_boxplot(df):
    # 차량 소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 승차 구역 리스트 생성
    global addr_list
    addr_list.columns = ['승차 구역명']
    addr_list_n = len(addr_list)
    # 차량소속별 분석 실행
    for t in car_type:
        if t != '전체':
            tmp = df[df['차량 소속'] == t]
        elif t == '전체':
            tmp = df            
        pickup_box = tmp[['차량 소속', '승차 구역명', '픽업 거리']]
        pickup_box = pickup_box[pickup_box['픽업 거리'].notnull()]     
        # 픽업 거리 데이터가 있는 경우만 분석 진행
        if len(pickup_box[pickup_box['픽업 거리'] == 0]) != len(pickup_box):               
            # 비어있는 승차 구역 추가 (승차 구역에 픽업거리 데이터가 없는 경우 0값 추가)
            pickup_addr = list(pickup_box['승차 구역명'].unique())
            if len(pickup_addr) != addr_list_n:
                for addr in addr_list['승차 구역명']:
                    if addr in pickup_addr:
                        continue
                    pickup_box = pickup_box.append(pd.DataFrame({'차량 소속': [t], '승차 구역명': [addr], '픽업 거리': [0]}))
            pickup_box.sort_values(['승차 구역명'], inplace = True)
            pickup_box.reset_index(drop = True, inplace = True)        
            # boxplot
            plt.figure(figsize = (20,15))
            sns.boxplot(x = "승차 구역명" , y = "픽업 거리", data = pickup_box, color = 'C0')
            plt.xlabel('승차 구역명', fontsize = 15, labelpad = 20)
            plt.ylabel('픽업 거리', fontsize = 15, labelpad = 20)
            plt.xticks(rotation = 90)
            plt.title('[특별교통수단 운영 현황 - 승차구역별 픽업 거리_' + t, fontsize = 25, pad = 20, loc = 'left')
            plot_name = 'EDA_승차 구역별 픽업 거리 분석_' + t + '_BoxPlot.png'
            plt.savefig(eda_dir_num + plot_name)
            print('[승차구역별 픽업 거리 분석_' + t + '_Boxplot] \n' + eda_dir_num + plot_name + '\n')
            plt.close()
        
        
# 콜접수 연월별 승차 구역별 대기시간 boxplot
def call_ym_boxplot(df):
    # 차량 소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 승차 구역 리스트 생성
    global addr_list
    addr_list.columns = ['승차 구역명']
    addr_list_n = len(addr_list)
    # 차량별 분석 실행
    for t in car_type:
        if t != '전체':
            tmp = df[df['차량 소속'] == t]
        elif t == '전체':
            tmp = df
        # 필요 컬럼 선택
        ym_box = tmp[['콜접수 연월', '승차 구역명', '대기 시간']]
        ym_box.sort_values(['콜접수 연월', '승차 구역명'], inplace = True)
        # 비어있는 연월 + 승차 구역 추가 (연월 + 승차 구역에 대기 시간 데이터가 없는 경우 0값 추가)
        for ym in list(ym_box['콜접수 연월'].unique()):
            tmp_ym = ym_box[ym_box['콜접수 연월'] == ym]
            tmp_ym_addr = list(tmp_ym['승차 구역명'].unique())
            if len(tmp_ym_addr) != addr_list_n:
                for addr in addr_list['승차 구역명']:
                    if addr in tmp_ym_addr:
                        continue
                    ym_box = ym_box.append(pd.DataFrame({'콜접수 연월': [ym], '승차 구역명': [addr], '대기 시간': [0]}))
        ym_box.sort_values(['콜접수 연월', '승차 구역명'], inplace = True)
        ym_box.reset_index(drop = True, inplace = True)
        # boxplot
        fig = px.box(ym_box, x = '승차 구역명', y = '대기 시간', range_y = [- 5, 105], animation_frame = '콜접수 연월')
        fig.update_layout(showlegend = False)
        plot_name = 'EDA_연월별 승차 구역별 대기 시간_' + t + '_BoxPlot.html'
        pio.write_html(fig, eda_dir_num + plot_name, auto_play = False)
        print('[연월별 승차 구역별 대기 시간_' + t + '] \n' + eda_dir_num + plot_name + '\n')
        
        
# 콜접수 시각별 대기시간 boxplot
def time_waiting_boxplot(df):
    # 차량 소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 시간 리스트 생성
    global time_list
    time_list.columns = ['콜접수 시각']
    time_list_n = len(time_list)
    # 차량소속별 분석 실행
    for t in car_type:
        if t != '전체':
            tmp = df[df['차량 소속'] == t]
        elif t == '전체':
            tmp = df   
        # 필요 컬럼 선택
        time_box = tmp[['차량 소속', '콜접수 시각', '대기 시간']]   
        # 비어있는 시각 추가 (해당 시각에 대기 시간 데이터가 없는 경우 0값 추가)
        box_list = list(time_box['콜접수 시각'].unique())
        if len(box_list) != time_list_n:
            for h in time_list['콜접수 시각']:
                if h in box_list:
                    continue
                time_box = time_box.append(pd.DataFrame({'차량 소속': [t], '콜접수 시각': [h], '대기 시간': [0]}))
        time_box.sort_values('콜접수 시각', inplace = True)
        time_box.reset_index(drop = True, inplace = True)        
        # summary
        summary_result = pd.DataFrame()
        for h in time_list['콜접수 시각']:
            tmp_table = time_box[time_box['콜접수 시각'] == h]
            tmp_summary = pd.DataFrame(tmp_table['대기 시간'].describe())
            tmp_summary.columns = ['콜접수 시각_' + str(h)]
            summary_result = pd.concat([summary_result, tmp_summary], axis = 1)
        summary_result = summary_result.loc[['count', 'min', '25%', '50%', 'mean', '75%', 'max']]
        summary_result.index = ['count', 'min', 'q1', 'median', 'mean', 'q3', 'max']
        # save
        table_name = 'EDA_콜접수 시각별 대기 시간_' + t + '_Summary.csv'
        summary_result.to_csv(eda_dir_num + table_name, encoding = 'MS949')
        print('[콜접수 시각별 대기 시간_' + t + '] \n' + eda_dir_num + table_name)       
        # boxplot
        plt.figure(figsize = (20,15))
        sns.boxplot(x = "콜접수 시각", y = "대기 시간", data = time_box, color = 'C0')
        plt.xlabel('콜접수 시각', fontsize = 15, labelpad = 20)
        plt.ylabel('대기 시간', fontsize = 15, labelpad = 20)
        plt.title('콜접수 시각별 대기 시간 boxplot_' + t, fontsize = 25, pad = 20, loc = 'left')
        plot_name = 'EDA_콜접수 시각별 대기 시간_' + t +'.Boxplot.png'
        plt.savefig(eda_dir_num + plot_name)
        print(eda_dir_num + plot_name + '\n')
        plt.close()
        
        
# 콜접수 요일별 대기시간 boxplot
def week_waiting_boxplot(df):    
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    for t in car_type:
        if t != '전체':
            tmp = df[df['차량 소속'] == t]
        elif t == '전체':
            tmp = df               
        week_box = tmp[['차량 소속', '콜접수 요일', '대기 시간']]
        week_box['콜접수 요일'] = pd.Categorical(week_box['콜접수 요일'], weekday_mapping.values())
        week_box.sort_values('콜접수 요일', inplace = True)
        week_box.reset_index(drop = True, inplace = True)    
        # summary
        summary_result = pd.DataFrame()
        for w in list(week_box['콜접수 요일'].unique()):
            tmp_table = week_box[week_box['콜접수 요일'] == w]
            tmp_summary = pd.DataFrame(tmp_table['대기 시간'].describe())
            tmp_summary.columns = ['콜접수 요일_' + w]
            summary_result = pd.concat([summary_result, tmp_summary], axis = 1)
        summary_result = summary_result.loc[['count', 'min', '25%', '50%', 'mean', '75%', 'max']]
        summary_result.index = ['count', 'min', 'q1', 'median', 'mean', 'q3', 'max']
        # save
        table_name = 'EDA_콜접수 요일별 대기 시간_' + t + '_Summary.csv'
        summary_result.to_csv(eda_dir_num + table_name, encoding = 'MS949')
        print('[콜접수 요일별 대기 시간_' + t + '] \n' + eda_dir_num + table_name)
        # boxplot        
        plt.figure(figsize = (20,15))
        sns.boxplot(x = "콜접수 요일" , y = "대기 시간", data = week_box, color = 'C0')
        plt.xlabel('콜접수 요일', fontsize = 15, labelpad = 20)
        plt.ylabel('대기 시간', fontsize = 15, labelpad = 20)        
        plt.title('[특별교통수단 운영 현황 - 콜접수 요일별 대기 시간_' + t, fontsize = 25, pad = 20, loc = 'left')
        plot_name = 'EDA_콜접수 요일별 대기 시간_' + t + '_BoxPlot.png'
        plt.savefig(eda_dir_num + plot_name)
        print(eda_dir_num + plot_name + '\n')
        plt.close()
        
        
# 콜접수 일자특성별 대기시간 boxplot
def calendar_waiting_boxplot(df):
    # 차량소속 리스트 생성
    car_type = ['전체'] + df['차량 소속'].unique().tolist()
    # 차량소속별 분석 실행
    for t in car_type:
        if t != '전체':
            tmp = df[df['차량 소속'] == t]
        elif t == '전체':
            tmp = df
        # 필요 컬럼 선택
        calendar_box = tmp[['차량 소속', '콜접수 일자 특성', '대기 시간']]
        calendar_box['콜접수 일자 특성'] = pd.Categorical(calendar_box['콜접수 일자 특성'], calendar_mapping.values())
        calendar_box.sort_values('콜접수 일자 특성', inplace = True)
        calendar_box.reset_index(drop = True, inplace = True)        
        # summary
        summary_result = pd.DataFrame()
        for c in list(calendar_box['콜접수 일자 특성'].unique()):
            tmp_table = calendar_box[calendar_box['콜접수 일자 특성'] == c]
            tmp_summary = pd.DataFrame(tmp_table['대기 시간'].describe())
            tmp_summary.columns = ['콜접수 일자 특성_' + c]
            summary_result = pd.concat([summary_result, tmp_summary], axis = 1)
        summary_result = summary_result.loc[['count', 'min', '25%', '50%', 'mean', '75%', 'max']]
        summary_result.index = ['count', 'min', 'q1', 'median', 'mean', 'q3', 'max']
        # save
        table_name = 'EDA_콜접수 일자 특성별 대기 시간_' + t + '_Summary.csv'
        summary_result.to_csv(eda_dir_num + table_name, encoding = 'MS949')
        print('[콜접수 일자 특성별 대기 시간_' + t + '] \n' + eda_dir_num + table_name)        
        # boxplot        
        plt.figure(figsize = (20,15))
        sns.boxplot(x = "콜접수 일자 특성" , y = "대기 시간", data = calendar_box, color = 'C0')
        plt.xlabel('콜접수 일자 특성', fontsize = 15, labelpad = 20)
        plt.ylabel('대기 시간', fontsize = 15, labelpad = 20)
        plt.title('[특별교통수단 운영 현황 - 콜접수 일자 특성별 대기 시간_' + t, fontsize = 25, pad = 20, loc = 'left')
        plot_name = 'EDA_콜접수 일자 특성별 대기 시간_' + t + '_BoxPlot.png'
        plt.savefig(eda_dir_num + plot_name)
        print(eda_dir_num + plot_name + '\n')
        plt.close()
        
        
# 대기시간 분석 (콜접수 일시가 50% 이상인 경우 실시)
def waiting_time(df):
    waiting_table = df[df['콜접수 일시'] != datetime.datetime(1900, 1, 1, 0, 0, 0)]
    if len(waiting_table) >= len(df) / 2:
        call_ym_boxplot(waiting_table)
        time_waiting_boxplot(waiting_table)
        week_waiting_boxplot(waiting_table)
        calendar_waiting_boxplot(waiting_table)

# 5. EDA

5-1. 이동구분별 운행 거리 분석

In [27]:
distance_result(data)

[운행 거리 분석_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_운행 거리_전체_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_운행 거리_전체_Plot.png

[운행 거리 분석_내부 이동] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_운행 거리_내부 이동_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_운행 거리_내부 이동_Plot.png



5-2. 장애유형별 운행 수 분석

In [28]:
disorder_table(data)

[장애 유형 분석] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_장애 유형별 운행 수 Table.csv



5-3. 차량번호별 운행 분석

In [29]:
car_table(data)

[차량 번호별 운행 내역 분석_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_차량 번호별 운행 내역_전체_Table.csv

[차량 번호별 운행 내역 분석_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_차량 번호별 운행 내역_센터차량_Table.csv

[차량 번호별 운행 내역 분석_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_차량 번호별 운행 내역_임차택시_Table.csv



5-4. 연월별 운행 분석 (차량소속 구분)

In [30]:
month_result(data)

[승차 연월별 운행 분석_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 연월별 운행 분석_전체_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 연월별 운행 분석_전체_Plot.png

[승차 연월별 운행 분석_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 연월별 운행 분석_센터차량_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 연월별 운행 분석_센터차량_Plot.png

[승차 연월별 운행 분석_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 연월별 운행 분석_임차택시_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 연월별 운행 분석_임차택시_Plot.png



5-5. 시각별, 요일별, 일자특성별 운행 분석 (차량소속 구분)

In [31]:
run_result(data)

[승차 시각별 운행 분석_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 시각별 운행 분석_전체_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 시각별 운행 분석_전체_Plot.png

[승차 시각별 운행 분석_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 시각별 운행 분석_센터차량_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 시각별 운행 분석_센터차량_Plot.png

[승차 시각별 운행 분석_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 시각별 운행 분석_임차택시_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 시각별 운행 분석_임차택시_Plot.png

[승차 요일별 운행 분석_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 운행 분석_전체_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 운행 분석_전체_Plot.png

[승차 요일별 운행 분석_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 운행 분석_센터차량_Table.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 운행 분석_센터차량_Plot.png

[승차 요일별 운행 분석_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/2

In [32]:
box_result(data)

[승차 요일별 운행 수 특성 분석_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 일별 운행 수 특성 분석_전체_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 일별 운행 수 특성 분석_전체_BoxPlot.png

[승차 요일별 운행 수 특성 분석_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 일별 운행 수 특성 분석_센터차량_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 일별 운행 수 특성 분석_센터차량_BoxPlot.png

[승차 요일별 운행 수 특성 분석_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 일별 운행 수 특성 분석_임차택시_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 요일별 일별 운행 수 특성 분석_임차택시_BoxPlot.png

[승차 일자 특성별 운행 수 특성 분석_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 일자 특성별 일별 운행 수 특성 분석_전체_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 일자 특성별 일별 운행 수 특성 분석_전체_BoxPlot.png

[승차 일자 특성별 운행 수 특성 분석_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 일자 특성별 일별 운행 수 특성 분석_센터차량_Summary.csv
C:/19Pro

5-7. 승차구역별 픽업 거리 분석

In [33]:
pickup_boxplot(data)

[승차구역별 픽업 거리 분석_전체_Boxplot] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 구역별 픽업 거리 분석_전체_BoxPlot.png

[승차구역별 픽업 거리 분석_센터차량_Boxplot] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 구역별 픽업 거리 분석_센터차량_BoxPlot.png

[승차구역별 픽업 거리 분석_임차택시_Boxplot] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_승차 구역별 픽업 거리 분석_임차택시_BoxPlot.png



5-8. 대기 시간 분석 (콜접수 일시가 전체 운행 데이터의 50% 이상 존재하는 경우 실행)

In [34]:
waiting_time(data)

[연월별 승차 구역별 대기 시간_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_연월별 승차 구역별 대기 시간_전체_BoxPlot.html

[연월별 승차 구역별 대기 시간_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_연월별 승차 구역별 대기 시간_센터차량_BoxPlot.html

[연월별 승차 구역별 대기 시간_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_연월별 승차 구역별 대기 시간_임차택시_BoxPlot.html

[콜접수 시각별 대기 시간_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_콜접수 시각별 대기 시간_전체_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_콜접수 시각별 대기 시간_전체.Boxplot.png

[콜접수 시각별 대기 시간_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_콜접수 시각별 대기 시간_센터차량_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_콜접수 시각별 대기 시간_센터차량.Boxplot.png

[콜접수 시각별 대기 시간_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_콜접수 시각별 대기 시간_임차택시_Summary.csv
C:/19Project/05.TRV/01.Pred_Demand/2. EDA_20211006_1/EDA_콜접수 시각별 대기 시간_임차택시.Boxplot.png

[콜접수 요일별 대기 시간_전체] 
C:/19Project/05.TRV/01.Pred_Demand/2. E